In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
import os
import pickle
import numpy as np
import json

In [3]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()

In [7]:
""" keras.layers.convolutional.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', 
                                      activation=None, 
                                      use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', 
                                      kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None)"""

model.add(Conv2D(32, (3, 3), padding='same',strides = (2,2), input_shape=x_train.shape[1:], use_bias=True )) #32 feature maps with a size of 3*3
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [8]:
model.add(Conv2D(64, (3, 3), padding='same')) #64 feature maps with a size of 3*3
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [9]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [10]:
# initiate SGD optimizer
sgd = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9)

In [11]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [13]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

Using real-time data augmentation.


In [14]:
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [15]:
# Fit the model on the batches generated by datagen.flow(). Used filter size of 2*2
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/10
1562/1562 [==============================] - 63s - loss: 1.6056 - acc: 0.4156 - val_loss: 1.3368 - val_acc: 0.5212
Epoch 2/10
1562/1562 [==============================] - 60s - loss: 1.2664 - acc: 0.5479 - val_loss: 1.0989 - val_acc: 0.6093
Epoch 3/10
1562/1562 [==============================] - 60s - loss: 1.1596 - acc: 0.5952 - val_loss: 1.0756 - val_acc: 0.6356
Epoch 4/10
1562/1562 [==============================] - 60s - loss: 1.1173 - acc: 0.6097 - val_loss: 0.9512 - val_acc: 0.6759
Epoch 5/10
1562/1562 [==============================] - 60s - loss: 1.1061 - acc: 0.6229 - val_loss: 1.0514 - val_acc: 0.6384
Epoch 6/10
1562/1562 [==============================] - 60s - loss: 1.1140 - acc: 0.6224 - val_loss: 0.9342 - val_acc: 0.6931
Epoch 7/10
1562/1562 [==============================] - 59s - loss: 1.1520 - acc: 0.6161 - val_loss: 1.1073 - val_acc: 0.6488
Epoch 8/10
1562/1562 [==============================] - 59s - loss: 1.1751 - acc: 0.6086 - val_loss: 1.0213 - val_acc:

In [16]:
# Filter size of 2*2 ans strides (2,2)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/10
1562/1562 [==============================] - 58s - loss: 1.6623 - acc: 0.3954 - val_loss: 1.5514 - val_acc: 0.4643
Epoch 2/10
1562/1562 [==============================] - 57s - loss: 1.3129 - acc: 0.5342 - val_loss: 1.1195 - val_acc: 0.6084
Epoch 3/10
1562/1562 [==============================] - 56s - loss: 1.2048 - acc: 0.5762 - val_loss: 0.9853 - val_acc: 0.6513.5
Epoch 4/10
1562/1562 [==============================] - 56s - loss: 1.1597 - acc: 0.5983 - val_loss: 1.0049 - val_acc: 0.6736
Epoch 5/10
1562/1562 [==============================] - 56s - loss: 1.1480 - acc: 0.6068 - val_loss: 0.9828 - val_acc: 0.6543
Epoch 6/10
1562/1562 [==============================] - 56s - loss: 1.1469 - acc: 0.6098 - val_loss: 0.9226 - val_acc: 0.6903
Epoch 7/10
1562/1562 [==============================] - 55s - loss: 1.1580 - acc: 0.6078 - val_loss: 1.1525 - val_acc: 0.6108
Epoch 8/10
1562/1562 [==============================] - 55s - loss: 1.1804 - acc: 0.6027 - val_loss: 1.1054 - val_ac

In [15]:
# Fit the model on the batches generated by datagen.flow(). Used filter size of 3*3 and activation function of 'relu' and 'sigmoid'
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/10
1562/1562 [==============================] - 139s - loss: 1.6668 - acc: 0.3889 - val_loss: 1.2337 - val_acc: 0.5571
Epoch 2/10
1562/1562 [==============================] - 134s - loss: 1.3163 - acc: 0.5337 - val_loss: 1.0655 - val_acc: 0.6223
Epoch 3/10
1562/1562 [==============================] - 133s - loss: 1.2025 - acc: 0.5808 - val_loss: 1.0603 - val_acc: 0.6259
Epoch 4/10
1562/1562 [==============================] - 134s - loss: 1.1635 - acc: 0.6007 - val_loss: 0.9142 - val_acc: 0.6773
Epoch 5/10
1562/1562 [==============================] - 131s - loss: 1.1401 - acc: 0.6104 - val_loss: 0.9240 - val_acc: 0.6838
Epoch 6/10
1562/1562 [==============================] - 121s - loss: 1.1496 - acc: 0.6106 - val_loss: 0.9453 - val_acc: 0.6750
Epoch 7/10
1562/1562 [==============================] - 133s - loss: nan - acc: 0.2630 - val_loss: nan - val_acc: 0.1000
Epoch 8/10
1562/1562 [==============================] - 174s - loss: nan - acc: 0.1000 - val_loss: nan - val_acc: 0.1

In [ ]:
# Save model and weights
def save_model(model):
 
   model_json = model.to_json()
   open('cifar10_architecture.json', 'w').write(model_json)
   model.save_weights('cifar10_weights.h5', overwrite=True)

In [17]:
# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)


In [17]:
# Evaluate model with test data set and share sample prediction results
# Results for 32 and 64 filters of size 2*2, stride of 1 and padding = same (zeros).

evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Model Accuracy = 0.62
Actual Label = cat vs. Predicted Label = frog
Actual Label = ship vs. Predicted Label = airplane
Actual Label = ship vs. Predicted Label = airplane
Actual Label = airplane vs. Predicted Label = cat
Actual Label = frog vs. Predicted Label = deer
Actual Label = frog vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = frog
Actual Label = frog vs. Predicted Label = cat
Actual Label = cat vs. Predicted Label = automobile
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = airplane vs. Predicted Label = cat
Actual Label = truck vs. Predicted Label = frog
Actual Label = dog vs. Predicted Label = ship
Actual Label = horse vs. Predicted Label = dog
Actual Label = truck vs. Predicted Label = deer
Actual Label = ship vs. Predicted Label = bird
Actual Label = dog vs. Predicted Label = frog
Actual Label = horse vs. Predicted Label = bird
Actual Label = ship vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label = frog


In [18]:
# Evaluate model with test data set and share sample prediction results
# Results for 32 and 64 filters of size 2*2, stride of 1 and padding = valid (No padding.

evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Model Accuracy = 0.64
Actual Label = cat vs. Predicted Label = frog
Actual Label = ship vs. Predicted Label = airplane
Actual Label = ship vs. Predicted Label = airplane
Actual Label = airplane vs. Predicted Label = frog
Actual Label = frog vs. Predicted Label = deer
Actual Label = frog vs. Predicted Label = dog
Actual Label = automobile vs. Predicted Label = frog
Actual Label = frog vs. Predicted Label = cat
Actual Label = cat vs. Predicted Label = automobile
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = airplane vs. Predicted Label = bird
Actual Label = truck vs. Predicted Label = frog
Actual Label = dog vs. Predicted Label = ship
Actual Label = horse vs. Predicted Label = dog
Actual Label = truck vs. Predicted Label = deer
Actual Label = ship vs. Predicted Label = airplane
Actual Label = dog vs. Predicted Label = frog
Actual Label = horse vs. Predicted Label = bird
Actual Label = ship vs. Predicted Label = automobile
Actual Label = frog vs. Predicted Labe